# Monte Carlo methods and finite size analysis in paramagnetic-ferromagnetic models of  phase transitions


###  Description and assignments

The objective is to study, by Monte Carlo simulations,  Ising-like modes on the square lattice  and look at  the behaviour of observables such as the specific heat, magnetisation and magnetic susceptibility, $\chi$,  as a function of the temperature ed in particular in proximity of the critical point (exactly known in $d=2$ due to the Onsager solution)

The Monte Carlo algorithm  can be based on proposed local moves of Glauber type (i.e. spin flip) and accepted with respect to the Metropolis-Hastings filter.
It would be interesting to see how the above mentioned observables behaves  as a function of the system size and perform a finite-size scaling analysis to estimate some critical exponents such as the gamma  exponent that governs  the divergence of chi at $T_c$.

A discussion on how to collect uncorrelated data and on the critical slowing down problem can be also considered.

### Contacts
* Enzo Orlandini <enzo.orlandini@unipd.it>
